In [4]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression , SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC


from nltk.classify import ClassifierI
from statistics import mode

class VoteClassifier(ClassifierI):
    def __init__(self, * classifiers):
        self._classifiers = classifiers
    
    def classify(self , features) :
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidance(self , features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
    

documents = [(list(movie_reviews.words(fileid)) , category)
            for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)

all_words = []
    
for w in movie_reviews.words():
    all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:3000]

def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

find_features(movie_reviews.words('neg/cv000_29416.txt'))

featuresets = [(find_features(rev) , category)
              for (rev , category) in documents]


training_set = featuresets[:1900]
testing_set = featuresets[1900:]

# clf = nltk.NaiveBayesClassifier.train(training_set)
clf_file = open("naivebayes.pickle", "rb") # load from pickle
clf = pickle.load(clf_file)
clf_file.close()

acc = nltk.classify.accuracy(clf , testing_set)

print("nltk naive bayes  Acc is " +str( acc*100))
# clf.show_most_informative_features(15) # 15 most informative features


# MNB classifiers
MNB_clf = SklearnClassifier(MultinomialNB())
MNB_clf.train(training_set)
acc = nltk.classify.accuracy(MNB_clf , testing_set)
print("MNB Acc is " +str( acc*100))
# clf.show_most_informative_features(15) # 15 most informative features

# Bernoulli classifiers
BNB_clf = SklearnClassifier(BernoulliNB())
BNB_clf.train(training_set)
acc = nltk.classify.accuracy(BNB_clf , testing_set)
print("BNB Acc is " +str( acc*100))
# clf.show_most_informative_features(15) # 15 most informative features

# LogisticRegression classifiers
LogisticRegression_clf = SklearnClassifier(LogisticRegression())
LogisticRegression_clf.train(training_set)
acc = nltk.classify.accuracy(LogisticRegression_clf , testing_set)
print("LogisticRegression  Acc is " +str( acc*100))

# SGDClassifier classifiers
SGDClassifier_clf = SklearnClassifier(SGDClassifier())
SGDClassifier_clf.train(training_set)
acc = nltk.classify.accuracy(SGDClassifier_clf , testing_set)
print("SGDClassifier Acc is " +str( acc*100))

# SVC classifiers
SVC_clf = SklearnClassifier(SVC())
SVC_clf.train(training_set)
acc = nltk.classify.accuracy(SVC_clf , testing_set)
print("SVC Acc is " +str( acc*100))

# LinearSVC_clf classifiers
LinearSVC_clf = SklearnClassifier(LinearSVC())
LinearSVC_clf.train(training_set)
acc = nltk.classify.accuracy(LinearSVC_clf , testing_set)
print("LinearSVC_clf Acc is " +str( acc*100))

# NuSVC classifiers
NuSVC_clf = SklearnClassifier(NuSVC())
NuSVC_clf.train(training_set)
acc = nltk.classify.accuracy(NuSVC_clf , testing_set)
print("NuSVC Acc is " +str( acc*100))

# voted classifiers
voted_clf = VoteClassifier(clf ,
                           MNB_clf ,
                           LogisticRegression_clf ,
                           SGDClassifier_clf,
                           LinearSVC_clf,
                           NuSVC_clf)

acc = nltk.classify.accuracy(voted_clf , testing_set)
print("voted Acc is " +str( acc*100))



print("Classification: ", voted_clf.classify(testing_set[0][0]) , "Confidance : " , voted_clf.confidance(testing_set[0][0]) )
print("Classification: ", voted_clf.classify(testing_set[1][0]) , "Confidance : " , voted_clf.confidance(testing_set[1][0]) )
print("Classification: ", voted_clf.classify(testing_set[2][0]) , "Confidance : " , voted_clf.confidance(testing_set[2][0]) )
print("Classification: ", voted_clf.classify(testing_set[3][0]) , "Confidance : " , voted_clf.confidance(testing_set[3][0]) )
print("Classification: ", voted_clf.classify(testing_set[4][0]) , "Confidance : " , voted_clf.confidance(testing_set[4][0]) )


nltk naive bayes  Acc is 92.0
MNB Acc is 84.0
BNB Acc is 83.0
LogisticRegression  Acc is 75.0


/home/sonu/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier Acc is 72.0
SVC Acc is 80.0
LinearSVC_clf Acc is 74.0
NuSVC Acc is 76.0


StatisticsError: no unique mode; found 2 equally common values